In [1]:
# 사용 모듈
from re import X
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import io
import imageio
import cv2
from glob import glob
from PIL import Image as img
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 

import pandas as pd
import codecs, json
import argparse
from pathlib import Path

from keras.models import load_model
from keras import backend as K

In [68]:
# # 사용 이미지
# import glob
# IMG_PATH = Path('D:/data/1.Training_crop_merge')
#   # 근대만 모은 폴더

# # 생장 데이터
# DATASET_PATH = Path('D:/data/1.Training/labels_merge')

# # 환경 데이터
# DATACSV_PATH = Path('D:/data/1.Training/labels_merge')

In [89]:
# 사용 이미지
import glob

# grop으로 이미지 불러오기
img_file = r'E:/data/1.Training_crop_merge/*.jpg'
IMG = glob.glob(img_file)


# 생장 데이터
wdata_file = r'E:/data/1.Training/labels_merge/*.json'
DATASET = glob.glob(wdata_file)

# 환경 데이터
sdata_file = r'E:/data/1.Training/labels_merge/*json'
DATACSV = glob.glob(sdata_file)

In [90]:
with open("img_file.txt", "w") as f:
    for ls in IMG:
        f.write(ls)
        f.write("\n")

with open("wdata_file.txt", "w") as f:
    for ls in DATASET:
        f.write(ls)
        f.write("\n")

with open("sdata_file.txt", "w") as f:
    for ls in DATACSV:
        f.write(ls)
        f.write("\n")

In [2]:
IMG_PATH ='E:/data/img_file.txt'
DATASET_PATH = 'E:/data/wdata_file.txt'
DATACSV_PATH = 'E:/data/sdata_file.txt'

In [3]:
read_trainset = False

In [4]:
# dataset 저장할 때 사용하는 함수

def write_dataset(fname, wdataset, idataset, sdataset ) :
  ## fname 은 저장할 txt파일 save_cashe + '/train_data.txt'
  ## nia 는 한국지능정보사회진흥원에 보낼 데이터들

    with open(fname, "w") as wfile:
        for idx,value in enumerate(idataset) :
            niadata = { "leaf_length" : wdataset[idx], "image" : value, "env" : sdataset[idx] }
            # For test
            # for idximg in value:
            #     print(f'images : { str(IMG_PATH) + "/" + idximg.split("/")[-1]}')
            #     img = cv2.imread( str(IMG_PATH) +'/' + idximg.split('/')[-1] )
            #     cv2.imwrite('./wdataset/temp/'+ str(wdataset[idx]) + '_' + idximg.split('/')[-1],cv2.resize(img,(400,400)))
            # sdataset에는 기존 코드에 있던 생장정보가 아닌 환경 정보 넣기
            # wdataset은 무게가 아닌 잎 길이 정보 넣기
            wfile.write(str(niadata) + '\n')

In [5]:
# 잎 찾기 함수
# 소스코드에서 사용된 적 없음

def detect_leaf(imgfile):

    img = cv2.imread(imgfile)

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # find the green color 
    # mask_green = cv2.inRange(hsv, (36,0,0), (86,255,255))

    # find the brown color
    mask_brown = cv2.inRange(hsv, (8, 60, 20), (30, 255, 200))
    # find the yellow and green color in the leaf
    mask_yellow_green = cv2.inRange(hsv, low_green, high_green)
    # find any of the three colors(green or brown or yellow) in the image

    mask = cv2.bitwise_or(mask_yellow_green, mask_brown)
    
    # mask = cv2.bitwise_or(mask, mask_yellow_green)
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(img, img, mask=mask)

    cv2.imshow("original", img)
    cv2.imshow("final image", res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return res



In [6]:
# 이미지 마스크
# 마스크 함수 이용 안 하고 전처리 해놓은 이미지 사용할 예정
# 다른 함수 속에 들어가서 실행되는 거라면 코드 수정해줘야함
RESIZE=64
def img_mask( imgfile, masked ) :
#     이미지 불러게만 바꿔줌


    imgorg = green = cv2.imread(imgfile)
#     if masked :
#         hsv = cv2.cvtColor(imgorg, cv2.COLOR_BGR2HSV)

#         # find the green color 
#         mask_green = cv2.inRange(hsv, (25,0,117), (171,194,192))
#         # find the brown color
#         mask_brown = cv2.inRange(hsv, (19, 60, 112), (179, 186, 222))
#         # find the yellow color in the leaf
#         mask_yellow = cv2.inRange(hsv, (36, 0, 0), (136, 255, 255))
#         # hsv = cv2.morphologyEx(hsv, cv2.MORPH_CLOSE, kernel=np.ones((11,11),dtype=np.uint8))
#         # apply mask to original image
#         # green = cv2.bitwise_and(imgorg, imgorg,mask=mask)

#         mask = cv2.bitwise_or(mask_green, mask_brown)
#         mask = cv2.bitwise_or(mask, mask_yellow)

#         result = cv2.bitwise_and(imgorg, imgorg, mask=mask)

#         result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel=np.ones((11,11),dtype=np.uint8))

#         cv2.imshow("original", imgorg)
#         cv2.imshow("middle image", hsv)
#         cv2.imshow("final image", result)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

    return green

def img_mask3( imgfile, masked ) :

    low_green = np.array([155,0,40])
    high_green = np.array([179,165,255])
    low_red = np.array([42,127,81])
    high_red= np.array([179,254,255])

    imgorg = green = cv2.imread(imgfile)
    if masked :
        hsv = cv2.cvtColor(imgorg, cv2.COLOR_BGR2HSV)

        # hsv = cv2.morphologyEx(hsv, cv2.MORPH_CLOSE, kernel=np.ones((11,11),dtype=np.uint8))
        # apply mask to original image
        # green = cv2.bitwise_and(imgorg, imgorg,mask=mask)

        mask = cv2.bitwise_not(cv2.inRange(hsv, low_green, high_green))
        mask2 = cv2.bitwise_not(cv2.inRange(hsv, low_red, high_red))

        mask =cv2.bitwise_not(mask+mask2)
        ## slice the green

        result = cv2.bitwise_and(imgorg, imgorg, mask=mask)
        result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel=np.ones((11,11),dtype=np.uint8))

        cv2.imshow("original", imgorg)
        cv2.imshow("middle image", hsv)
        cv2.imshow("final image", result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return result

def img_mask2( imgfile, masked ) :
    imgorg = green = cv2.imread(imgfile)
    if masked :

        # imgorg = cv2.adaptiveThreshold(imgorg,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,15,2)

        low_green = np.array([20,144,88])
        high_green = np.array([167,247,164])

        mask = cv2.inRange(imgorg, (86,66,0), (190,190,255))
        cv2.imshow("1", mask)

        mask2 = cv2.bitwise_not(mask)
        cv2.imshow("2", mask2)

        # output = cv2.bitwise_and(imgorg,imgorg, mask= mask)

        hsv = cv2.cvtColor(imgorg, cv2.COLOR_BGR2HSV)

        # hsv = cv2.morphologyEx(hsv, cv2.MORPH_CLOSE, kernel=np.ones((11,11),dtype=np.uint8))
        # apply mask to original image
        # green = cv2.bitwise_and(imgorg, imgorg,mask=mask)

        mask = cv2.inRange(hsv, low_green, high_green)
        ## slice the green

        result = cv2.bitwise_and(imgorg, imgorg, mask=mask)
        result2 = cv2.bitwise_and(result, result, mask=mask2)

        result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel=np.ones((11,11),dtype=np.uint8))
        result2 = cv2.morphologyEx(result2, cv2.MORPH_CLOSE, kernel=np.ones((11,11),dtype=np.uint8))

        cv2.imshow("original", imgorg)
        cv2.imshow("final image", result2)
        cv2.imshow("middle image", result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return result

In [7]:
# 소스코드에 사용된 적 없음

def kmeans_color_quantization(image, clusters=8, rounds=1):
    h, w = image.shape[:2]
    samples = np.zeros([h*w,3], dtype=np.float32)
    count = 0

    for x in range(h):
        for y in range(w):
            samples[count] = image[x][y]
            count += 1

    compactness, labels, centers = cv2.kmeans(samples,
            clusters, 
            None,
            (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10000, 0.0001), 
            rounds, 
            cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers)
    res = centers[labels.flatten()]
    return res.reshape((image.shape))



In [8]:
import numpy as np
import pandas as pd

path = './1209.npy'
data = np.load(path, allow_pickle=True)
data = pd.DataFrame(data)
data.columns=['filename_mod', 'create_date', 'crops', 'kind_type', 'fname', 'growth_stage', 'receive_date', 
              'temp_min', 'humi_min', 'co2_min', 'temp_max', 'humi_max', 'co2_max', 'temp_exp', 'humi_exp', 'co2_exp', 
              'ir_value', 'tl_value', 'ei_value', 'pl_value', 'measured_date', 
 'stem_length', 'leaf_cnt', 'leaf_width', 'leaf_length', 'stem_thick', 'fr_weight', 'cameranum']

data

,filename_mod,create_date,crops,kind_type,fname,growth_stage,receive_date,temp_min,humi_min,co2_min,...,ei_value,pl_value,measured_date,stem_length,leaf_cnt,leaf_width,leaf_length,stem_thick,fr_weight,cameranum
0,C26_L04_01_001_439336.jpg,2021-11-08,근대,청경근대,C26_L04_01_001_210914213908.jpg,생육기,2021-09-14 21:38:54,21.0,50.0,255.0,...,0.14,4.5,2021-09-14,5.9,2,2.5,3.4,0,0,C26_L04_01_001
1,C26_L04_01_001_439874.jpg,2021-11-08,근대,청경근대,C26_L04_01_001_210923213908.jpg,생육기,2021-09-23 21:38:42,21.0,54.0,184.0,...,0.16,4.7,2021-09-23,13.0,5,3.5,4.2,0,0,C26_L04_01_001
2,C26_L04_01_001_559451.jpg,2021-12-01,근대,청경근대,C26_L04_01_001_210927213908.jpg,수확기,2021-09-27 21:38:11,21.0,59.0,179.0,...,0.17,4.7,2021-09-27,16.0,6,5.9,7.2,0,8.0,C26_L04_01_001
3,C26_L04_01_002_439009.jpg,2021-11-08,근대,청경근대,C26_L04_01_002_210907213908.jpg,생육기,2021-09-07 21:38:56,21.0,50.0,275.0,...,0.12,4.5,2021-09-07,6.2,2,2.0,3.3,0,0,C26_L04_01_002
4,C26_L04_01_002_439331.jpg,2021-11-08,근대,청경근대,C26_L04_01_002_210914133908.jpg,생육기,2021-09-14 13:38:59,19.0,51.0,265.0,...,0.14,4.8,2021-09-14,11.0,2,2.8,3.7,0,0,C26_L04_01_002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,C29_L04_02_009_551466.jpg,2021-11-25,근대,청경근대,C29_L04_01_009_211019171257.jpg,정식기,2021-10-19 17:12:24,19.0,48.0,394.0,...,0.18,4.2,2021-10-19,2.4,2,0.9,1.3,0,0,C29_L04_02_009
243,C29_L04_02_009_552167.jpg,2021-11-25,근대,청경근대,C29_L04_01_009_211026170959.jpg,생육기,2021-10-26 17:09:17,21.0,48.0,428.0,...,0.20,3.9,2021-10-26,4.7,4,1.1,1.5,0,0,C29_L04_02_009
244,C29_L04_02_009_553050.jpg,2021-11-25,근대,청경근대,C29_L04_01_009_211102160959.jpg,생육기,2021-11-02 16:09:47,21.0,47.0,385.0,...,0.22,4.0,2021-11-02,6.2,4,1.7,2.7,0,0,C29_L04_02_009
245,C29_L04_02_009_553890.jpg,2021-11-25,근대,청경근대,C29_L04_01_009_211109160959.jpg,생육기,2021-11-09 16:09:38,21.0,48.0,336.0,...,0.25,3.9,2021-11-09,9,6,3,4.5,0,0,C29_L04_02_009


In [9]:
def read_dataset(opt): ## train_dataset, val_dataset, test_dataset, strain_dataset, sval_dataset, stest_dataset, wtrain_dataset, wval_dataset, wtest_dataset 구하기
    ## opt에서 계속 가져옴
    img_source = str(opt.source) + '/'
      ## opt.source는 IMG_PATH
    save_cashe = str(opt.save_cashe)
      ## opt.save_cashe는 DATASET_PATH
    if not opt.cashe :
        columnz = ["cameranum","measured_date","filename_mod","fr_weight","stem_length","leaf_cnt","leaf_length","leaf_width"]
          ## 사용한 columns
          ## crop_id 는 cameranum
          ## 우리에 맞게 바꿔주기
          ## day_capture에서 measured_date
          ## 기존 json의 path는 image와 맞지 않았기 때문에 image와 맞게 새로 만든 filename_mod
          ## weight에서 fr_weigth
        dataFrame = pd.read_csv( opt.data, names=columnz).loc[1:,:]
          ## 우리는 csv 말고 npy 사용할 예정
        cnt = 0
          ## 왜 개수를 셀까??

        dataset, oneframex, imageset, idata =  None, None, None, None
          ## dataset은 np.expand_dims(oneframex, axis=0)
          ## dataset은 np.concatenate([dataset, np.expand_dims(oneframex, axis=0)])


          ## oneframex는 imageset[index-opt.window:index,...]

          ## imagesetdms np.concatenate([imageset, idata])

          ## idata는 np.array([np.array(cv2.resize(mask,(opt.imgsz,opt.imgsz)))])

        sdataset,oneframes, sensorset, sdata =  None, None, None, None
          ## sdataset는 np.array([np.array([float(row['stem_length']),float(row['leaf_cnt']),float(row['leaf_length']),float(row['leaf_width'])])])
          ## sdataset는 np.concatenate([sdataset, np.array([np.array([float(row['stem_length']),float(row['leaf_cnt']),float(row['leaf_length']),float(row['leaf_width'])])])])

          ## oneframes, sensorset, sdata는 없음
          ## 우리 데이터는 어떤 걸로 해야할지 고민

        wdataset = np.array([])

        nia_dataset, nia_oneframex, nia_imageset = None, None, np.full((20),None)  #제출용
          ## nia_imageset은 np.full((20),None)
          ## none값인 애들을 20행 만들기

        for index, row in dataFrame.iterrows():
          ## iterrows 메서드는 데이터의 행-열/데이터 정보를 튜플 형태의 generator 객체로 반환하는 메서드
          ## index랑 row 가져옴
            if not pd.isnull(row['leaf_length']):
                cnt = cnt +1
                print( f"' index : {index-opt.window} -- cameranum : {dataFrame.loc[index-opt.window,'cameranum']} -- leaf_length : {dataFrame.loc[index-opt.window,'leaf_length']}")
                  ## loc로 crops_id랑 weight column 가져옴
                print( f"' index : {index} -- cameranum : {dataFrame.loc[index,'cameranum']} -- leaf_length : {dataFrame.loc[index,'leaf_length']}")
                  ## loc로 crops_id랑 weight column 가져옴

                mask = img_mask (img_source+row['filename_mod'].split('/')[-1], True)
                  ## img_source는 str(opt.source) + '/'
                  ## opt.source는 image path
                  ## [-1]은 이미지 파일 이름일듯

                # cv2.imwrite('./wdataset/temp/'+row['paths'].split('/')[-1],cv2.resize(mask,(opt.imgsz,opt.imgsz)))

                idata = np.array([np.array(cv2.resize(mask,(opt.imgsz,opt.imgsz)))])
                  ## image resize 부분
                  ##  resize는 위에서 64로 지정해줌
                  ## image 가져온 걸 nparray로 변환해줌
                  ## np.array(np.array[])로 차원 추가해줌 이미 3차원에 차원을 추가하면 (개수, 64, 64, 2) 이렇게 되는 건가?
                  ## idata는 상위에서 만든 지역변수임


                imageset = np.concatenate([imageset, idata])
                  ## imageset은 아직 none인 상태였음
                  ## nparray 상태인 이미지를 가져옴


                nia_imageset = np.concatenate([nia_imageset, np.expand_dims(np.array(row['paths']), axis=0)])
                  ## none값인 애들을 20행 만들기
                  ## path는 이미지 이름인데 왜 가져오는지 모르겠음
                  ## np.expend_dims는 차원을 추가함 axis=0임으로 행을 추가하기 때문에 1행 n열이 됨
                  ## 우선 이미지를 가져왔다고 치기


                oneframex = imageset[index-opt.window:index,...]
                  ## window 는 time steps로 봄 default는 10으로 되어있음
                  ## index-window : index는 10개를 계산해서 답을 도출한다는 뜻
                  ## 어떻게 10개를 가져오는지 모르겠음 수정해볼 필요가 있음

                nia_oneframex = nia_imageset[index-opt.window:index,...]
                  ## nia_oneframex는 상위에서 None으로 선언함

                if dataFrame.loc[index-opt.window,'cameranum'] == dataFrame.loc[index,'cameranum']:
                    if row['leaf_length'] == '`' : 
                        row['leaf_length'] = np.nan
                          ## none값 nan으로 대체
                    if row['leaf_width'] == '`' : 
                        row['leaf_width'] = np.nan
                          ## none값 nan으로 대체
                    if dataset is None : 
                        sdataset = np.array([np.array([float(row['temp_min']),float(row['humi_min']),float(row['co2_min']),float(row['temp_max']),float(row['humi_max']),float(row['co2_max']),float(row['temp_exp']),float(row['humi_exp']),float(row['co2_exp']),float(row['ir_value']),float(row['tl_value']),float(row['ei_value']),float(row['pl_value'])])])
                        # oneframez = np.expand_dims(oneframe, axis=0)
                          ## dataset 상위에서 none으로 선언함
                          ## sdataset 상위에서 none으로 선언함
                          ## 생장정보들만 가져옴
                        dataset = np.expand_dims(oneframex, axis=0)
                          ## oneframex = imageset[index-opt.window:index,...]
                          ## 한번에 학습시킬 이미지들 가져와서 행 추가
                        nia_dataset = np.expand_dims(nia_oneframex, axis=0)
                          ## nia_dataset 상위에서 none으로 선언함
                          ## nia_dataset은 nia_imageset[index-opt.window:index,...]로 지정함
                          ## dataset이랑 nia_dataset 차이를 모르겠음

                    else :
                        sdataset = np.array([np.array([float(row['temp_min']),float(row['humi_min']),float(row['co2_min']),float(row['temp_max']),float(row['humi_max']),float(row['co2_max']),float(row['temp_exp']),float(row['humi_exp']),float(row['co2_exp']),float(row['ir_value']),float(row['tl_value']),float(row['ei_value']),float(row['pl_value'])])])
                            # oneframez = np.concatenate([oneframez, np.expand_dims(oneframe, axis=0)])
                          ## 상위에서 sdata는 none값이었음, 여기서는 if 아닐 경우 else로해서 기존 sdataset에 concatenate 해줌

                        dataset = np.concatenate([dataset, np.expand_dims(oneframex, axis=0)])
                        nia_dataset = np.concatenate([nia_dataset, np.expand_dims(nia_oneframex, axis=0)])
                    wdataset = np.concatenate([wdataset, np.array([float(dataFrame.loc[index,'leaf_length'])])])
                      ## wdataset은 위에서 np.array([])로 선언
                    print( f' index {index}  oneframex shape {oneframex.shape} wdataset shape {wdataset.shape}  dataset shape {dataset.shape} sdataset shape {sdataset.shape}')
              ## 여기까지가 if not pd.isnull(row['weight']):일 경우
            else :
              ## weigt가 null일 경우들
                print ( img_source + '/' + row['filename_mod'].split('/')[-1] )


                mask = img_mask (img_source+row['filename_mod'].split('/')[-1], True)
                  ## 위에서도 똑같이 mask = img_mask(img_source + row['paths'].split('/')[-1], True)

                # cv2.imwrite('./wdataset/temp/'+row['paths'].split('/')[-1],cv2.resize(mask,(opt.imgsz,opt.imgsz)))

                idata = np.array([np.array(cv2.resize(mask,(opt.imgsz,opt.imgsz)))])
                  ## 위에서도 똑같이 idata = np.array([np.array(cv2.resize(mask,(opt.imgsz,opt.imgsz)))])

                if imageset is None :
                    imageset = idata
                      ## image resize 부분
                      ##  resize는 위에서 64로 지정해줌
                      ## image 가져온 걸 nparray로 변환해줌
                      ## np.array(np.array[])로 차원 추가해줌 이미 3차원에 차원을 추가하면 (개수, 64, 64, 2) 이렇게 되는 건가?
                      ## idata는 상위에서 만든 지역변수임
                    nia_imageset = np.expand_dims(np.array(row['filename_mod']), axis=0)
                      ## none값인 애들을 20행 만들기
                      ## path는 이미지 이름인데 왜 가져오는지 모르겠음
                      ## np.expend_dims는 차원을 추가함 axis=0임으로 행을 추가하기 때문에 1행 n열이 됨
                      ## 우선 이미지를 가져왔다고 치기

                else :
                    imageset = np.concatenate([imageset, idata])
                      ## image resize 부분
                      ##  resize는 위에서 64로 지정해줌
                      ## image 가져온 걸 nparray로 변환해줌
                      ## np.array(np.array[])로 차원 추가해줌 이미 3차원에 차원을 추가하면 (개수, 64, 64, 2) 이렇게 되는 건가?
                      ## idata는 상위에서 만든 지역변수임
                    nia_imageset = np.concatenate([nia_imageset, np.expand_dims(np.array(row['filename_mod']), axis=0)])
                      ## none값인 애들을 20행 만들기
                      ## path는 이미지 이름인데 왜 가져오는지 모르겠음
                      ## np.expend_dims는 차원을 추가함 axis=0임으로 행을 추가하기 때문에 1행 n열이 됨
                      ## 우선 이미지를 가져왔다고 치기


        print( f'count = {cnt}')
          ## cnt로 계산함

        indexes = np.arange(dataset.shape[0])
          ## dataset 개수만큼
          ## idata = np.array([np.array(cv2.resize(mask, (opt.imgsz, opt.imgsz)))])
          ## imageset = np.concatenate([imageset, idata])
          ## oneframex = imageset[index-opt.window:index,...]
          ## dataset = np.expand_dims(oneframex, axis=0)

        np.random.shuffle(indexes)
          ## sdataset(길이), wdataset(무게)도 같은 index 사용
          ## 여기서는 lstm과 별개로 cnn만 돌리기 때문에 섞어도 됨

        train_index = indexes[: int(0.8 * dataset.shape[0])]
        val_index = indexes[int(0.8 * dataset.shape[0]) : int(0.9 * dataset.shape[0])]
        test_index = indexes[int(0.9 * dataset.shape[0]) :]
          ## index 를 train, val, test로 나눔

        train_dataset = dataset[train_index]
        val_dataset = dataset[val_index]
        test_dataset = dataset[test_index]

        strain_dataset = sdataset[train_index]
        sval_dataset = sdataset[val_index]
        stest_dataset = sdataset[test_index]

        # weight train set
        wtrain_dataset = wdataset[train_index]
        wval_dataset = wdataset[val_index]
        wtest_dataset = wdataset[test_index]

        write_dataset(save_cashe + '/train_data.txt',wtrain_dataset.tolist() , nia_dataset[train_index].tolist() ,strain_dataset.tolist())
        write_dataset(save_cashe + '/val_data.txt',wval_dataset.tolist() , nia_dataset[val_index].tolist() ,sval_dataset.tolist())
        write_dataset(save_cashe + '/test_data.txt',wtest_dataset.tolist() , nia_dataset[test_index].tolist() ,stest_dataset.tolist())
          ## 저장하기
          ## w, s, nia 한번에 저장
        np.save( save_cashe + '/train_dataset',train_dataset )
        np.save( save_cashe + '/val_dataset',val_dataset )
        np.save( save_cashe + '/test_dataset',test_dataset )
        np.save( save_cashe + '/strain_dataset',strain_dataset )
        np.save( save_cashe + '/sval_dataset',sval_dataset )
        np.save( save_cashe + '/stest_dataset',stest_dataset )

        np.save( save_cashe + '/wtrain_dataset',wtrain_dataset )
        np.save( save_cashe + '/wval_dataset',wval_dataset )
        np.save( save_cashe + '/wtest_dataset',wtest_dataset )
          ## 각각 .npy로 저장
        
    else:
          ## opt.cashe 있으면 불러오기
        train_dataset = np.load( save_cashe + '/train_dataset.npy' )
        val_dataset = np.load( save_cashe + '/val_dataset.npy' )
        test_dataset = np.load( save_cashe + '/test_dataset.npy' )
        strain_dataset = np.load( save_cashe + '/strain_dataset.npy' )
        sval_dataset = np.load( save_cashe + '/sval_dataset.npy' )
        stest_dataset = np.load( save_cashe + '/stest_dataset.npy' )

        wtrain_dataset = np.load( save_cashe + '/wtrain_dataset.npy' )
        wval_dataset = np.load( save_cashe + '/wval_dataset.npy' )
        wtest_dataset = np.load( save_cashe + '/wtest_dataset.npy' )


        print( f' train_dataset shape {train_dataset.shape} strain_dataset shape {strain_dataset.shape}  wtrain_dataset shape {wtrain_dataset.shape}')

    # Normalize the data to the 0-1 range.
      ## Normalization (정규화) 특성들을 특정 범위(주로 [0,1]) 로 스케일링 하는 것, 가작 작은 값은 0, 가장 큰 값은 1 
    train_dataset = train_dataset / 255
    val_dataset = val_dataset / 255
    test_dataset = test_dataset / 255

    scaler = StandardScaler()
      ## (표준화) 특성들의 평균을 0, 분산을 1 로 스케일링, 특성들을 정규분포로 만듬

    strain_dataset = scaler.fit_transform(strain_dataset)
      ## train은 .fit_transform 나머지는 .tranform
    sval_dataset = scaler.transform(sval_dataset)
    stest_dataset = scaler.transform(stest_dataset)

    strain_dataset = np.where(np.isnan(strain_dataset), 0, strain_dataset)
    sval_dataset = np.where(np.isnan(sval_dataset), 0, sval_dataset)
    stest_dataset = np.where(np.isnan(stest_dataset), 0, stest_dataset)
      ## 조건 맞는 곳의 특정 값 변경
      ## data에서 nan인 값들을 0으로 바꾸겠다

    return train_dataset, val_dataset, test_dataset, strain_dataset, sval_dataset, stest_dataset, wtrain_dataset, wval_dataset, wtest_dataset

In [10]:
# input할 데이터들 shape 바꿔주기

def create_shifted_frames(data):
    x = data[:, 0 : data.shape[1] - 1, :, :]
    #x = data[:, 9 : data.shape[1] - 1, :, :]
    y = data[:, -1 , :, :]
    return x, y

  ## image shifted로 shape 통일
  ## 값이 x, y 2개 나옴

def create_shifted_env(data):
    x = data[:, 0 : data.shape[1] - 1,:]
    #x = data[:, 9 : data.shape[1] - 1]
    y = data[:, -1,:]
    return x, y

In [11]:
# opt 지정
def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data', default = DATACSV_PATH, help='dataset.yaml path')
    parser.add_argument('--weights', nargs='+', type=str, default= DATASET_PATH, help='model.h5 path(s)')
    parser.add_argument('--batch', type=int, default=15, help='batch size')
    parser.add_argument('--epochs', type=int, default=100, help='epochs size')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=RESIZE, help='inference size (pixels)')
    parser.add_argument('--task', default='train', help='train, test, predict')
    parser.add_argument('--source', default= IMG_PATH, help='orignal crop information list or image for predict')
    parser.add_argument('--save_cashe', default= DATASET_PATH / 'vegetable_size', help=' train dataset location ')
    parser.add_argument('--window', default=10, help='lstm window')
    parser.add_argument('--type', default='vegetable', help='strawberry or vegetable')
    parser.add_argument('--cashe', action='store_true', help='read cashe dataset to train')
    parser.add_argument('--detect', action='store_true', help='detect dataset ')
    opt = parser.parse_args()
    return opt

In [12]:
!pip install easydict

  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6506 sha256=74a3cfe3019db945eacbb0ffd676caf1caa1191e7c8087b29bb7f7fc79b7e0f3
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\0d\9a\a9\02f3a5f0c6b2c57184661770360c58db8166f5c877780e98f2
Successfully built easydict


In [13]:
# import easydict
# def parse_opt():
#     args = easydict.EasyDict({
# # 
#             "data": DATACSV_PATH,

#             "weights": DATASET_PATH,

#             "batch": 15,

#             "epochs": "100",

#             'imgsz' : RESIZE,
        
#             'img' : RESIZE,
        
#             'img-size': RESIZE,

#             "task": 'train',
            
#             "source": IMG_PATH,
           
#             "save_cashe" : DATASET_PATH,
        
#             "window" : 10,
            
#             "type" : 'vegetable',
        
#             "cashe" : 'store_true',
        
#             "detect" : 'store_true'

#     })
    
# #     return opt

In [14]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [15]:
def main(opt):

    train_dataset, val_dataset, test_dataset, strain_dataset, sval_dataset, stest_dataset, wtrain_dataset, wval_dataset, wtest_dataset = read_dataset(opt)
    
    # Apply the processing function to the image datasets.
    x_train, y_train = create_shifted_frames(train_dataset)
    x_val, y_val = create_shifted_frames(val_dataset)
    x_test, y_test = create_shifted_frames(test_dataset)

    # Apply the processing function to the env datasets.
    x_strain = strain_dataset
    x_sval = sval_dataset
    x_stest = stest_dataset

    # Apply the processing function to the weight datasets.
    y_trainY = wtrain_dataset.reshape(1,-1)
    y_valY = wval_dataset.reshape(1,-1)
    y_testY = wtest_dataset.reshape(1,-1)

    y_trainY= np.reshape(y_trainY, (y_trainY.shape[1],1))
    y_valY= np.reshape(y_valY, (y_valY.shape[1],1))
    y_testY= np.reshape(y_testY, (y_testY.shape[1],1))

    sc = MinMaxScaler()
    # sc = StandardScaler()

    y_trainY[:] = sc.fit_transform(y_trainY[:])
    y_valY[:] = sc.transform(y_valY[:])
    y_testY[:] = sc.transform(y_testY[:])


    # output layer

    # Inspect the dataset.
    print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
    print("Validation Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape)) 


    # Construct a figure on which we will visualize the images.
    fig, axes = plt.subplots(4, 5, figsize=(10, 8))

    # Plot each of the sequential images for one random data example.
    data_choice = np.random.choice(range(len(train_dataset)), size=1)[0]
    for idx, ax in enumerate(axes.flat):
        if  train_dataset.shape[1] > idx :
            ax.imshow(np.squeeze(train_dataset[data_choice][idx]), cmap="gray")
            ax.set_title(f"Frame {idx + 1}")
            ax.axis("off")

    # Print information and display the figure.
    print(f"Displaying frames for example {data_choice}.")
    plt.show()

    """
    ## Model Construction

    """
        ## opt.type으로 할 일 정해줌

    if opt.task == 'train':

        # Construct the input layer with no definite frame size.
        inp = layers.Input(shape=(None, *x_train.shape[2:]), name='imgInput')
          ## image input으로 넣음

        # We will construct 3 `ConvLSTM2D` layers with batch normalization,
        # followed by a `Conv3D` layer for the spatiotemporal outputs.
        x = layers.ConvLSTM2D(
            filters=64,
            kernel_size=(5, 5),
            padding="same",
            return_sequences=True,
            activation="relu",
        )(inp)
        x = layers.BatchNormalization()(x)
        x = layers.ConvLSTM2D(
            filters=64,
            kernel_size=(3, 3),
            padding="same",
            return_sequences=True,
            activation="relu",
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.ConvLSTM2D(
            filters=64,
            kernel_size=(1, 1),
            padding="same",
            return_sequences=True,
            activation="relu",
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.ConvLSTM2D(
            filters=64,
            kernel_size=(2, 2),
            padding="same",
            return_sequences=False,
            activation="relu",
        )(x)
        x = layers.BatchNormalization()(x)
        xImg = layers.Conv2D(
            filters=3, kernel_size=(3, 3), activation="sigmoid", padding="same", name='next'
        )(x)

        # env model
        inp_env = layers.Input(shape=(x_strain.shape[1]), name='envInput')
          ## 환경 정보 input으로
        size = layers.Dense(500, activation='relu')(inp_env)
        size = layers.Dense(384, activation='relu')(size)

        xEnv_flattern = layers.Flatten()(size)
         ## image 정보 flatten 레이어

        # weight model
        y = layers.MaxPool2D(pool_size=32)(x)
        y = layers.Dense(84, activation='relu')(y)
        y = layers.Dense(30, activation='relu')(y)
        y = layers.Flatten()(y)

        # merge image + env
        y = layers.concatenate([y, xEnv_flattern])
          ## 여기서 image랑 환경 정보 합침


        if opt.type == 'strawberry':        #=====> 딸기
            print("=================> 딸기 ")
            y = layers.Dense(512, activation='relu')(y)
            y = layers.Dropout(0.5)(y)
            y = layers.Dense(100, activation='relu')(y)
            y = layers.Dropout(0.5)(y)
            y = layers.Dense(25, activation='relu')(y)
            y = layers.Dense(1, name='leaf_length')(y)
        else:                               #=====> 엽채
            print("=================> 엽채 모델")
            y = layers.Dense(500, activation='relu')(y)
            y = layers.Dense(320, activation='relu')(y)
            y = layers.Dense(384, activation='relu')(y)
            y = layers.Dense(448, activation='relu')(y)
            y = layers.Dense(160, activation='relu')(y)
            y = layers.Dense(160, activation='relu')(y)
            y = layers.Dense(32, activation='relu')(y)
            y = layers.Dense(1, name='leaf_length')(y)

        # Next, we will build the complete model and compile it.
        model = keras.models.Model(inputs=[inp,inp_env], outputs=[y])
          ## 모델 input 정해주기

        model.summary()

        model.compile(
            loss={'leaf_length': root_mean_squared_error}, 
            # loss={'next':keras.losses.binary_crossentropy, 'env': keras.losses.mean_squared_error, 'weigth': keras.losses.mean_absolute_percentage_error}, 
 
            # [ keras.losses.binary_crossentropy, keras.losses.mean_squared_error, keras.losses.mean_squared_error], 
            optimizer=keras.optimizers.Adam(learning_rate=0.0001),
        )

        """
        ## Model Training

        With our model and data constructed, we can now train the model.
        """

        # Define some callbacks to improve training.
        early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

        # Define modifiable training hyperparameters.
        epochs = opt.epochs
        batch_size =opt.batch

        # Fit the model to the training data.
        model.fit( 
            { 'imgInput': x_train, 'envInput' : x_strain },
            { 'leaf_length': y_trainY },
            batch_size=batch_size,
            epochs=epochs,
            validation_data= ([x_val,x_sval], [y_valY]),
            # callbacks=[early_stopping, reduce_lr],
        )
          ## 모델 학습
        model.save( opt.weights )

    elif opt.task == 'test':
 
        model = load_model (opt.weights, custom_objects={'root_mean_squared_error': root_mean_squared_error})
        model.evaluate([x_test,x_stest],[y_testY], batch_size=opt.batch)

        print('test')

    elif opt.task == 'predict':

        # Select a random example from the validation dataset.
        # example = test_dataset[np.random.choice(range(len(val_dataset)), size=1)[0]]

        # Pick the first/last ten frames from the example.
        # frames = example[0:18, ...]

        # Predict a new set of 10 frames.
            # Extract the model's prediction and post-process it.
        model = load_model (opt.weights)
          ## keras에 있는 load모델 가져옴
        new_prediction = model.predict([x_test[0:10],x_stest[0:10]])

        print(mean_squared_error(y_testY[0:10]*100, new_prediction[2]*100))

        print((new_prediction[2]-y_testY[0:10])*100)
        print(new_prediction[2]*100)
        print(y_testY[0:10]*100)
        # print(y_testY[0:10])
            # Extend the set of prediction frames.

        # # Construct a figure for the original and new frames.
        # fig, axes = plt.subplots(2, 10, figsize=(20, 4))

        # # Plot the original frames.
        # for idx, ax in enumerate(axes[0]):
        #     ax.imshow(np.squeeze(test_dataset[0][idx]), cmap="gray")
        #     ax.set_title(f"Frame {idx }")
        #     ax.axis("off")

        # ax.imshow(np.squeeze(new_prediction), cmap="gray")
        # ax.set_title(f"Frame new")
        # ax.axis("off")

        # # Display the figure.
        # plt.show()

        # print('predict')


In [16]:
if __name__ == "__main__":
    opt = parse_opt()
    main(opt)


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [18]:
img_mask("C:\\Users\\pc\\test1.jpg",True)

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 44,  39,  41],
        [ 44,  39,  41],
        [ 44,  39,  41]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 44,  39,  41],
        [ 44,  39,  41],
        [ 44,  39,  41]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [ 44,  39,  41],
        [ 44,  39,  41],
        [ 44,  39,  41]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [200, 190, 190],
        [201, 191, 191],
        [201, 191, 191]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [200, 190, 190],
        [201, 191, 191],
        [201, 191, 191]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [200, 190, 190],
        [201, 191, 191],
        [201, 191, 191]]